## Základní připojení k API

### Zkopírujte si tento notebook do svého Google Drive

### Nastavte si API klíč

In [1]:
import instructor
import google.generativeai as genai
import os


genai.configure(api_key=os.environ["GEMINI_API_KEY"])
client = instructor.from_gemini(
    client=genai.GenerativeModel(
        model_name="models/gemini-1.5-flash-latest",
    ),
    mode=instructor.Mode.GEMINI_JSON,
)
test = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Hello, how are you?"},
    ],
)
print(test)


/Users/simon/Documents/czechitas-ai-data/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyError: 'GEMINI_API_KEY'